# More Practice With SQL Queries - Lab

## Introduction

In this lesson, we'll run through some practice questions to refresh your knowledge of SQL Queries!

## Objectives

You will be able to:

- Use `GROUP BY` statements in SQL to apply aggregate functions like: `COUNT`, `MAX`, `MIN`, and `SUM`
- Decide and perform whichever type of join is best for retrieving desired data
- Use the `HAVING` clause to compare different aggregates
- Write subqueries to decompose complex queries

## Getting Started

As in previous labs, we'll make use of the `sqlite3` library as well as Pandas. By combining them, we'll be able to write queries as Python strings, and make sure that the results are always returned as a Pandas DataFrame. 

We'll start by loading both libraries and connecting to the database we'll be using for this lab, `data.sqlite`. You may remember this database from a previous lab. As a refresher, here's the ERD diagram for this database: 

<img src='images/Database-Schema.png'>

In the cell below:

* Import the necessary libraries `pandas` and `sqlite3` 
* Establish a connection to the database `data.sqlite` 
* Get the `cursor` from the connection and store it in the variable `c` 

In [1]:
import sqlite3
import pandas as pd
conn =  sqlite3.connect('data.sqlite')
c = conn.cursor() 

## Basic Queries

Now, let's review basic SQL queries. In the cell below:

Write a query that gets the first name, last name, phone number, address, and credit limit for all customers in California with a credit limit greater than 25000.00. 

In [4]:
# For the first query, the boilerplate for getting 
# the query into a DataFrame has been provided for you
c.execute("""SELECT contactFirstName, contactLastName, phone, addressLine1, creditLimit
             FROM customers
             WHERE creditLimit > 25000
             ORDER BY creditLimit DESC
             ;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,contactFirstName,contactLastName,phone,addressLine1,creditLimit
0,Diego,Freyre,(91) 555 94 44,"C/ Moralzarzal, 86",227600
1,Susan,Nelson,4155551450,5677 Strong St.,210500
2,Mihael,Holz,0897-034555,Grenzacherweg 237,141300
3,Jeff,Young,2125557413,4092 Furth Circle,138500
4,Rachel,Ashworth,(171) 555-1555,Fauntleroy Circus,136800
...,...,...,...,...,...
89,Thomas,Smith,(171) 555-7555,120 Hanover Sq.,43300
90,William,Brown,2015559350,7476 Moss Rd.,43000
91,Juri,Yoshido,6175559555,8616 Spinnaker Dr.,41900
92,Yu,Choi,2125551957,5290 North Pendale Street,39800


#### Expected Output

<img src='images/expected-output-1.png'>

## Aggregate Functions and GROUP BY

Next, write a query that gets the average credit limit per state.

In [5]:
c.execute("""SELECT state, AVG(creditLimit)
             FROM customers
             GROUP BY state
             ORDER BY AVG(creditLimit) DESC
             ;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,state,AVG(creditLimit)
0,NH,114200.000000
1,NSW,100550.000000
2,Tokyo,94400.000000
3,Isle of Wight,93900.000000
4,NY,89966.666667
5,BC,89950.000000
6,Victoria,88800.000000
7,PA,84766.666667
8,CA,83854.545455
9,Osaka,81200.000000


#### Expected Output

<img src='images/expected-output-2.png'>

## JOINs

Now, write a query that uses JOIN statements to get the customer name, customer number, order number, status, and quantity ordered. Print only the head of this DataFrame. 

In [6]:
c.execute('''SELECT customerName, customerNumber, orderNumber, status, quantityOrdered
             FROM customers
             JOIN orders 
             USING(customerNumber)
             JOIN orderdetails
             USING(orderNumber)
             ;''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,customerName,customerNumber,orderNumber,status,quantityOrdered
0,Atelier graphique,103,10123,Shipped,26
1,Atelier graphique,103,10123,Shipped,34
2,Atelier graphique,103,10123,Shipped,46
3,Atelier graphique,103,10123,Shipped,50
4,Atelier graphique,103,10298,Shipped,32
...,...,...,...,...,...
2991,Kelly's Gift Shop,496,10399,Shipped,32
2992,Kelly's Gift Shop,496,10399,Shipped,40
2993,Kelly's Gift Shop,496,10399,Shipped,51
2994,Kelly's Gift Shop,496,10399,Shipped,57


#### Expected Output

<img src='images/joins.png' width='500' height='600'> 

## HAVING and ORDER BY

Now, return the customerName, customerNumber, productName, productCode, and total number ordered for any product a customer has bought 10 or more of cumulatively. Sort the rows in descending order by the quantity ordered. 

**_Hint_**: For this one, you'll need to make use of HAVING, GROUP BY, and ORDER BY -- make sure you get the order of them correct!

In [18]:
c.execute('''SELECT c.customerName, c.customerNumber, p.productName, p.productCode, sum(od.quantityOrdered) AS TotalOrdered
             FROM customers c
             JOIN orders o
             ON c.customerNumber = o.customerNumber
             JOIN orderdetails od
             ON od.orderNumber = o.orderNumber
             JOIN products p
             USING(productCode)
             GROUP BY c.customerNumber, productCode
             HAVING TotalOrdered > 10
             ORDER BY TotalOrdered DESC
             ;''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,customerName,customerNumber,productName,productCode,TotalOrdered
0,Euro+ Shopping Channel,141,1992 Ferrari 360 Spider red,S18_3232,308
1,Euro+ Shopping Channel,141,1958 Chevy Corvette Limited Edition,S24_2840,245
2,Euro+ Shopping Channel,141,1970 Dodge Coronet,S24_1444,197
3,Euro+ Shopping Channel,141,1957 Chevy Pickup,S12_4473,183
4,Euro+ Shopping Channel,141,2002 Chevy Corvette,S24_3432,174
...,...,...,...,...,...
2524,L'ordine Souveniers,386,P-51-D Mustang,S18_2581,15
2525,"Tokyo Collectables, Ltd",398,1936 Mercedes Benz 500k Roadster,S24_3969,15
2526,The Sharp Gifts Warehouse,450,1969 Chevrolet Camaro Z28,S24_3191,13
2527,La Rochelle Gifts,119,1954 Greyhound Scenicruiser,S32_2509,11


In [19]:
c.execute("""SELECT c.customerName, c.customerNumber, p.productName,
                    p.productCode, sum(od.quantityOrdered) as TotalOrdered
                    FROM Customers c JOIN Orders o 
                                     ON c.customerNumber = o.customerNumber
                                     JOIN OrderDetails od
                                     ON od.orderNumber = o.orderNumber
                                     JOIN Products p
                                     USING(productCode)
             GROUP BY c.customerNumber, productCode
             HAVING SUM(od.quantityOrdered) >= 10 
             ORDER BY TotalOrdered DESC;"""
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerName,customerNumber,productName,productCode,TotalOrdered
0,Euro+ Shopping Channel,141,1992 Ferrari 360 Spider red,S18_3232,308
1,Euro+ Shopping Channel,141,1958 Chevy Corvette Limited Edition,S24_2840,245
2,Euro+ Shopping Channel,141,1970 Dodge Coronet,S24_1444,197
3,Euro+ Shopping Channel,141,1957 Chevy Pickup,S12_4473,183
4,Euro+ Shopping Channel,141,2002 Chevy Corvette,S24_3432,174


#### Expected Output

<img src='images/having_order.png'>

## Subqueries

Finally, get the first name, last name, employee number, and office code for employees from offices with less than 5 employees. Print the first five rows of this DataFrame. 

In [20]:
c.execute('''SELECT lastName, firstName, employeeNumber, officeCode
             FROM employees
             WHERE officeCode IN (SELECT officeCode
                                  FROM offices
                                  JOIN employees
                                  USING(officeCode)
                                  GROUP BY officeCode
                                  HAVING COUNT(employeeNumber) < 5)
             ;''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,lastName,firstName,employeeNumber,officeCode
0,Patterson,William,1088,6
1,Firrelli,Julie,1188,2
2,Patterson,Steve,1216,2
3,Tseng,Foon Yue,1286,3
4,Vanauf,George,1323,3


#### Expected Output

<img src='images/expected-output-5.png'>

# Summary

In this lesson, we reviewed all the major concepts and keywords associated with SQL queries!